## Feature Selection

References: <br>
<a href='https://www.kaggle.com/tilii7/recursive-feature-elimination/comments'>Kaggle notebook</a> <br>
<a href='https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html#sphx-glr-auto-examples-feature-selection-plot-rfe-with-cross-validation-py'>Sklearn RFECV Page</a>

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold,KFold,train_test_split,GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error as mae
import matplotlib.pyplot as plt
import gc
import time

## Prepare data frame of the extracted features

In [2]:
# Load target feature
train=pd.read_csv('inputs/train.csv')
loss=train.loss
del train
gc.collect()

# load engineered features
features=pd.read_csv('inputs/features.csv')
features=features[features.columns[1:]]

# Downcast dataframe to save RAM.
# credit:https://www.kaggle.com/anshuls235/m5-forecasting-eda-fe-modelling
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

start=time.time()
features=downcast(features)
loss=downcast(pd.DataFrame(loss))
end=time.time()
print('Execution time in minutes:',(end-start)/60)
loss=np.ravel(loss)

Execution time in minutes: 21.717430444558463


## A) Recurrsive feature elimination with cross validation (RFECV) <br>
Recursive feature elimination (RFE) method is a recurrsive method to eliminate least improtant features. It does so by calculating feature importance of the predictor features by predicting target values using a user defined machine learning model. At every step RFE class removes the least important feature untill a user defined number of features are left. On the other hand recurrsive feture elimination with cross validation (RFECV) automatically selects optimal number of features by making use of the cross validation score.

#### Estimator:
For this problem we use random forest regressor(RFR) as our estimator. Paramter of RFR are:
<ol>
    <li>Number of estimators (decision trees): 100</li>
    <li>Maximum depth of any tree: 12</li>
    <li>Maximum number of features when looking for the best split: $\text{n_features}^{1/2}$</li>
</ol>

#### RFECV:
<ol>
    <li>Estimator: Random forest regressor</li>
    <li>Number of features to eliminate at each iteration: 10</li>
    <li>Cross validation method: 5 fold</li>
    <li>Scoring funciton: Mean absolute error</li>
</ol>

In [ ]:
# Define the estimator: random forest regressor
rfr = RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=12, n_jobs=-1)

In [ ]:
rfecv = RFECV(estimator=rfr,
              step=10,
              cv=KFold(n_splits=10,
                       shuffle=False),
              scoring='neg_mean_absolute_error',
              verbose=2,
             n_jobs=-1)

start=time.time()
rfecv.fit(features,(loss+1)**0.25)
end=time.time()
print('Execution time in hrs:',(end-start)/3600)

### Model performance for number of features

In [ ]:
print(' Optimal number of features: %d' % rfecv.n_features_)
fig,ax=plt.subplots(figsize=(12,5))
ax.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
ax.set_xlabel('Number of features tested x 10')
ax.set_ylabel('Cross-validation score (negative MAE)',fontsize=10)
plt.savefig('Allstate-RFECV.png')
plt.show();

### Save feature selection data frame

In [ ]:
all_features=features.columns
sel_features = [f for f, s in zip(all_features, rfecv.support_) if s]
ranking = pd.DataFrame({'Features': all_features})
ranking['Rank'] = np.asarray(rfecv.ranking_)
ranking.sort_values('Rank', inplace=True)
ranking.to_csv('inputs/Allstate-RFECV-ranking.csv', index=False)
print(' Ranked features saved:  Allstate-RFECV-ranking.csv')
score = round(-np.max(rfecv.grid_scores_), 3)
print('Best score:',score)